In [8]:
# Step 1: loading all required libraries
import pandas as pd
import numpy as np
from rapidfuzz import fuzz, process
from sklearn.metrics import precision_score, recall_score, f1_score


In [9]:
# Load your customer dataset (Kaggle or otherwise)
customer_df = pd.read_csv("C:/Users/ajaxc/Downloads/Sanctioned_List_By_Countries.csv")
input_names = customer_df["name"].dropna().unique().tolist()
input_names


['VALUMAR LLC',
 'Hadi Soleimanpour',
 'Alí Fallahijan',
 'Ahmad Reza Asghari',
 'hussein mohamed jomma',
 'inbrahin fadi ibrahin',
 'BARAKAT ALI HASSAN',
 'Iullia GILFANOV Iuliia GILFANOVA Yulia GILFANOVA Yuliya Raisovna GILFANOVA',
 'Aldo SANCHEZ',
 'HUSSEIN AHMAD YOUSSEF',
 'Mechanic Industries Group',
 'Aleksandar Radovanović',
 'Berislav Pušić',
 'Bogdan Grba',
 'Boris Breberina',
 'Boro Trbojević',
 'Bozo Latinovic',
 'Brane Džinić',
 'Branislav Medan',
 'Damir Sireta',
 'Darko Perić',
 'Dragan Borojević',
 'Dragan Medić',
 'Dragan Stanojević',
 'Djuro Kljajic',
 'Dusan Kovacevic',
 'Eva Radeka',
 'Idhan Sipić',
 'Ilija Vukelic',
 'Ivica Vrdoljak',
 'Jadranko Palija',
 'Jovan Dimitrijević',
 'Marko Škrobić',
 'Milan Devčić',
 'Milan Jaric',
 'Milan Ogrizovic',
 'Mile Ikac',
 'Milenko Trifunović',
 'Miloš Radić',
 'Miodrag Dimitrijević',
 'Miodrag Šolaja',
 'Mirko Pekez',
 'Mirko Todorović',
 'Goran Mugoša',
 'Nenad Bizic',
 'Nenad Tanasković',
 'Nikola Andrun',
 'Nikola Golubovic

In [10]:
# Load OFAC SDN list and rename second column to 'entity_name'
ofac_df = pd.read_csv("C:/Users/ajaxc/Downloads/sdn.csv")
cols = list(ofac_df.columns)
cols[1] = "entity_name"
ofac_df.columns = cols
sanctions_list = ofac_df["entity_name"].dropna().unique().tolist()
sanctions_list


['ANGLO-CARIBBEAN CO., LTD.',
 'BANCO NACIONAL DE CUBA',
 'BOUTIQUE LA MAISON',
 'CASA DE CUBA',
 'CECOEX, S.A.',
 'CIMEX',
 'CIMEX IBERICA',
 'CIMEX, S.A.',
 'COMERCIAL IBEROAMERICANA, S.A.',
 'COMERCIAL CIMEX, S.A.',
 'COMERCIAL DE RODAJES Y MAQUINARIA, S.A.',
 'COMERCIALIZACION DE PRODUCTOS VARIOS',
 'COMPANIA DE IMPORTACION Y EXPORTACION IBERIA',
 'CORPORACION CIMEX, S.A.',
 'COTEI',
 'CRYMSA - ARGENTINA, S.A.',
 'CUBACANCUN CIGARS AND GIFT SHOPS',
 'CUBAEXPORT',
 'CUBAFRUTAS',
 'CUBAN CIGARS TRADE',
 'CUBANATUR',
 'CUBATABACO',
 'CUMEXINT, S.A.',
 'DELVEST HOLDING, S.A.',
 'EDICIONES CUBANAS',
 'EDYJU, S.A.',
 'EMPRESA CUBANA DE AVIACION',
 'EMPRESA CUBANA DE PESCADOS Y MARISCOS',
 'EMPRESA DE TURISMO NACIONAL Y INTERNACIONAL',
 'ETCO INTERNATIONAL COMMODITIES LTD.',
 'ETCO INTERNATIONAL COMPANY, LIMITED',
 'GALAX TRADING CO., LTD.',
 'HAVIN BANK LIMITED',
 'HAVANATUR',
 'INSTITUTO NACIONAL DE TURISMO DE CUBA',
 'INTERCONSULT',
 'KAVE, S.A.',
 'LA EMPRESA CUBANA DE FLETES',
 'LEVE

In [ ]:
#input_names[1]

'Hadi Soleimanpour'

In [ ]:
#sanctions_list[11504]

'DAOUD, Suleiman Cabdi'

In [ ]:
# threshold = 85
# result = process.extractOne(
#         input_names[1],
#         sanctions_list,
#         scorer=fuzz.token_sort_ratio
#     )
# result



('DAOUD, Suleiman Cabdi', 57.89473684210527, 11504)

In [ ]:
# if result:
#     best_match, score, _ = result
# else:
#     best_match, score, _ = None, 0, None
   

In [ ]:
# is_hit = score >= threshold
# is_hit

False

In [ ]:
# To find the best matching name and get its score
def fast_match(input_name: str, sanctions_list: list[str], threshold=85) -> tuple[str,int]:  
    """
    Use rapidfuzz.process.extractOne for fast approximate matching.
    Returns best match, score, and match flag.
    """
    result = process.extractOne(
        input_name,
        sanctions_list,
        scorer=fuzz.token_sort_ratio
    )
    
    if result:
        match_name, score, _ = result
        return match_name, score
    else:
        return None, 0

def fast_screen(inputs: list, sanctions_list: list, threshold=85) ->pd.DataFrame :
    """
    Fast fuzzy screening over list of input names.
    """
    results = []
    for name in inputs:
        best_match, score = fast_match(name, sanctions_list)
        is_hit = score >= threshold
        results.append({
            "Input Name": name,
            "Best Match": best_match,
            "Match Score": score,
            "Is Match": is_hit
        })
    return pd.DataFrame(results)


In [ ]:
# You can reduce the number of records during testing for speed
# input_names = input_names[:500]

df_screened = fast_screen(input_names, sanctions_list, threshold=85)
df_screened.head()


,Input Name,Best Match,Match Score,Is Match
0,VALUMAR LLC,UMAC LLC,73.684211,False
1,Hadi Soleimanpour,"DAOUD, Suleiman Cabdi",57.894737,False
2,Alí Fallahijan,"AZZAM, Mansour Fadlallah",52.631579,False
3,Ahmad Reza Asghari,"SHIVA'I, Ahmad Asghari",70.000000,False
4,hussein mohamed jomma,"SALAD, Mohamed Hussein",60.465116,False


In [59]:
def categorize(score):
    if score >= 85:
        return "Strong Match"
    elif score >= 60:
        return "Partial Match"
    elif score > 0:
        return "Weak Match"
    else:
        return "No Match"

df_screened["Match Category"] = df_screened["Match Score"].apply(categorize)


In [ ]:
df_screened.to_csv("C:/Users/ajaxc/Downloads/df_screened.csv", index=False)